# O objetivo da camada silver é:

- Receber dados da camada Bronze após um pré-processamento inicial, como limpeza, transformação e validação. 
- Pode ser considerada a "fonte da verdade" para um data lake, contendo a versão mais atualizada de um dado. 
- É um ponto de acesso mais orientado para o negócio, com dados refinados para uso em análises profundas.
- Dados ausentes de clima serão tratados.

In [1]:
import pandas as pd
import duckdb

# Extração

- Tabela Consumo
- Tabela Clima
- Tabela Clientes

In [2]:
# Caminho do banco
db_path = "../../data/duckdb/database.duckdb"

# Conexão com o banco DuckDB
con = duckdb.connect(db_path)

# Carrega os dados da camada bronze
clientes_df = con.execute("SELECT * FROM bronze.clientes").df()
consumo_df = con.execute("SELECT * FROM bronze.consumo").df()
clima_df = con.execute("SELECT * FROM bronze.clima ORDER BY region, date").df()

In [3]:
# Cria schema 'silver' se não existir
con.execute("CREATE SCHEMA IF NOT EXISTS silver")

# Transformação

In [4]:
# Interpolação dos valores nulos de temperature por região
concat_df = []
for region in clima_df['region'].drop_duplicates():
    df_region = clima_df[clima_df['region'] == region].copy()
    df_region = df_region.set_index('date')
    df_region['temperature'] = df_region['temperature'].interpolate(method='time')
    df_region['region'] = region
    df_region = df_region.reset_index()
    df_region['temperature'] = df_region['temperature'].bfill()
    concat_df.append(df_region)

clima_df = pd.concat(concat_df)

clima_df = clima_df[['region', 'date', 'temperature', 'humidity']]

print(clima_df.isna().sum())


region         0
date           0
temperature    0
humidity       0
dtype: int64


# Carregamento

In [5]:
# 🔸 Cria tabelas dentro do schema silver
con.execute("""
CREATE TABLE IF NOT EXISTS silver.clientes (
    client_id VARCHAR,
    region VARCHAR
)
""")

con.execute("""
CREATE TABLE IF NOT EXISTS silver.consumo (
    client_id VARCHAR,
    date DATE,
    consumption_kwh DOUBLE
)
""")

con.execute("""
CREATE TABLE IF NOT EXISTS silver.clima (
    region VARCHAR,
    date DATE,
    temperature DOUBLE,
    humidity DOUBLE
)
""")

In [6]:
# Limpa dados se as tabelas já existirem
con.execute("DELETE FROM silver.clientes")
con.execute("DELETE FROM silver.consumo")
con.execute("DELETE FROM silver.clima")

In [7]:
# Registra como tabelas temporárias
con.register("clientes_df", clientes_df)
con.register("consumo_df", consumo_df)
con.register("clima_df", clima_df)

In [8]:
# Insere os dados nas tabelas silver
con.execute("INSERT INTO silver.clientes SELECT * FROM clientes_df")
con.execute("INSERT INTO silver.consumo SELECT * FROM consumo_df")
con.execute("INSERT INTO silver.clima SELECT * FROM clima_df")

In [9]:
con.close()
print("✔ Dados inseridos com sucesso na camada silver do DuckDB.")

✔ Dados inseridos com sucesso na camada silver do DuckDB.
